In [1]:
%pip install -q llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-ollama 0.4.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import os
from dotenv import load_dotenv
load_dotenv()
import warnings
warnings.filterwarnings('ignore')

In [7]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [9]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["./upload/The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf"])
documents = reader.load_data()

In [10]:
len(documents)

740

In [11]:
documents[500].metadata

{'page_label': '464',
 'file_name': 'The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf',
 'file_path': 'upload/The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf',
 'file_type': 'application/pdf',
 'file_size': 12585336,
 'creation_date': '2024-12-09',
 'last_modified_date': '2024-12-08'}

In [12]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/740 [00:00<?, ?it/s]

In [13]:
len(nodes)

785

In [14]:
nodes[152].text

'HAN 10-ch03-083-124-9780123814791 2011/6/1 3:16 Page 115 #33\n3.5 Data Transformation and Data Discretization 115\nThus, z-score normalization using the mean absolute deviation is\nv′\ni=vi−¯A\nsA. (3.11)\nThe mean absolute deviation, sA, is more robust to outliers than the standard deviation,\nσA. When computing the mean absolute deviation, the deviations from the mean (i.e.,\n|xi−¯x|) are not squared; hence, the effect of outliers is somewhat reduced.\nNormalization by decimal scaling normalizes by moving the decimal point of values\nof attribute A. The number of decimal points moved depends on the maximum absolute\nvalue of A. A value, vi, ofAis normalized to v′\niby computing\nv′\ni=vi\n10j, (3.12)\nwhere jis the smallest integer such that max(|v′\ni|)< 1.\nExample 3.6 Decimal scaling. Suppose that the recorded values of Arange from−986 to 917. The\nmaximum absolute value of Ais 986. T o normalize by decimal scaling, we therefore\ndivide each value by 1000 (i.e., j=3) so that−986 

In [15]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
llm = Groq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY)
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/740 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/785 [00:00<?, ?it/s]

In [28]:
vectDB = "./Gen.AI/DB/textbookstorage"
vector_index.storage_context.persist(persist_dir=vectDB)
storage_context = StorageContext.from_defaults(persist_dir=vectDB)
index = load_index_from_storage(storage_context, service_context=service_context)
query_engine = index.as_query_engine(service_context=service_context)

In [31]:
query = "Explain cosine similarity"
resp = query_engine.query(query)

In [24]:
import textwrap

def Print(text, width=80, **args):
    lines = text.split('\n')  # Split the text into lines based on original line breaks
    wrapped_lines = []
    for line in lines:
        wrapped_lines.extend(textwrap.wrap(line, width=width))  # Wrap each line individually
    print('\n'.join(wrapped_lines), **args)



In [32]:
Print(resp.response)

Cosine similarity is a measure of similarity that can be used to compare
documents or vectors. It is defined as the cosine of the angle between two
vectors. The cosine similarity function is given by the dot product of the two
vectors divided by the product of their Euclidean norms.
Conceptually, it measures the cosine of the angle between two vectors. A cosine
value of 0 means that the two vectors are at 90 degrees to each other
(orthogonal) and have no match. The closer the cosine value to 1, the smaller
the angle and the greater the match between vectors.
In the context of term-frequency vectors, cosine similarity focuses on the words
that two documents have in common and the occurrence frequency of such words,
ignoring zero-matches. It is a nonmetric measure, meaning it does not obey all
the properties of a metric measure.
When attributes are binary-valued, the cosine similarity function can be
interpreted in terms of shared features or attributes. It measures the relative
possessi

In [33]:
for node in resp.source_nodes:
    # Access the TextNode object directly
    text_node = node.node

    # Assuming metadata is stored within the TextNode's metadata
    source = text_node.metadata.get('file_name') # Access metadata using .metadata.get()
    page = text_node.metadata.get('page_label')  # Access metadata using .metadata.get()

    Print(f"Source: {source}")
    Print(f"Page: {page}")

Source: The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-
Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-
Morgan-Kaufmann-2011.pdf
Page: 78
Source: The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-
Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-
Morgan-Kaufmann-2011.pdf
Page: 77


In [34]:
query = """Create concept check questions and answer about cosine similarity.
Format output in markdown with 2 line breaks to separate markdown entities"""
resp = query_engine.query(query)
Print(resp.response)

**Concept Check Question 1**
What is cosine similarity used for in data analysis?
**Answer**
Cosine similarity is a measure of similarity that can be used to compare
documents or give a ranking of documents with respect to a given vector of query
words.
**Concept Check Question 2**
What type of data does cosine similarity ignore when calculating similarity?
**Answer**
Cosine similarity ignores zero-matches when calculating similarity.
**Concept Check Question 3**
Why is cosine similarity useful for comparing term-frequency vectors?
**Answer**
Cosine similarity is useful for comparing term-frequency vectors because it
focuses on the words that the two documents do have in common, and the
occurrence frequency of such words.
**Concept Check Question 4**
What is the main advantage of using cosine similarity over traditional distance
measures for sparse numeric data?
**Answer**
The main advantage of using cosine similarity is that it ignores zero-matches,
which makes it more suitable for sp

In [35]:
for node in resp.source_nodes:
    # Access the TextNode object directly
    text_node = node.node

    # Assuming metadata is stored within the TextNode's metadata
    source = text_node.metadata.get('file_name') # Access metadata using .metadata.get()
    page = text_node.metadata.get('page_label')  # Access metadata using .metadata.get()

    Print(f"Source: {source}")
    Print(f"Page: {page}")

Source: The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-
Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-
Morgan-Kaufmann-2011.pdf
Page: 77
Source: The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-
Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-
Morgan-Kaufmann-2011.pdf
Page: 269
